In [173]:
## SKLEARN
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.compose import make_column_transformer

import numpy as np

np.set_printoptions(suppress=True)
import pandas as pd
from platform import python_version

import random

# works, sort of only. Possible additional commas that shouldn't be there.
%load_ext nb_black

print("Libs imported. Python version is: ", python_version())

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
Libs imported. Python version is:  3.9.7


<IPython.core.display.Javascript object>

In [172]:
# utility functions

cols_mushroom = [
    "labels",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat",
]

cols_car = ["buying", "maintenance", "doors", "passengers", "boot", "safety", "labels"]

cols_audiology = [
    "age_gt_60",
    "air",
    "airBoneGap",
    "ar_c",
    "ar_u",
    "bone",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "o_ar_c",
    "o_ar_u",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "speech",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
    "p-index",
    "labels",
]

cols_wine = [
    "labels",
    "alcohol",
    "mallic-acid",
    "alcalinity",
    "ash",
    "magnesium",
    "total-phenols",
    "flavonids",
    "nonflavonid-phenols",
    "proanthocyanins",
    "color-intensity",
    "hue",
    "od-of-diluted-wines",
    "proline",
]

"""
https://archive.ics.uci.edu/ml/datasets/car+evaluation
0-5 -> data
6 -> labels
"""


def load_car():
    df_car = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
        header=None,
        names=cols_car,
    )
    # mappings using indexes:
    # X = df_car.loc[:, :5].values
    # y = df_car.loc[:, 6].values
    labels_col = df_car.pop("labels")
    df_car.insert(0, "labels", labels_col)
    return df_car


"""
https://archive.ics.uci.edu/ml/datasets/mushroom
1-22 -> data
0 -> labels
"""


def load_mushroom():
    df_mushroom = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
        header=None,
        names=cols_mushroom,
    )
    # index mappings
    # X = df_mushroom.loc[:, 1:].values
    # y = df_mushroom.loc[:, 0].values
    return df_mushroom


"""
https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29
0:length-2 -> data
length-1 unique id (p1-p200)
length -> labels
"""


def load_audiology():
    df_audiology = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/audiology/audiology.standardized.data",
        header=None,
        names=cols_audiology,
    )
    # index mapping
    # length = len(df_audiology.columns)
    # X = df_audiology.loc[:, : length - 3].values
    # y = df_audiology.loc[:, length - 1].values
    df_audiology = df_audiology.drop("p-index", axis=1)
    labels_col = df_audiology.pop("labels")
    df_audiology.insert(0, "labels", labels_col)
    return df_audiology


"""
https://www.alldatascience.com/classification/wine-dataset-analysis-with-python/
1:length -> data
0 -> labels
"""


def load_wine():
    df_wine = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",
        header=None,
        names=cols_wine,
    )
    # index mappings
    # length = len(df_wine.columns)
    # X = df_wine.loc[:, 1:].values
    # y = df_wine.loc[:, 0].values
    return df_wine

<IPython.core.display.Javascript object>

In [171]:
# Choose dataset
dataset = load_car()
# dataset = load_mushroom()
# dataset = load_audiology()
# dataset = load_wine()

print(dataset.info())
print("First five records:")
print(dataset.head())

# Extract to X and y
X_cat = dataset.loc[:, dataset.columns != "labels"]
y_cat = dataset.loc[:, "labels"]

print("Size of X: ", np.shape(X_cat))
print("Size of y: ", np.shape(y_cat))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   labels       1728 non-null   object
 1   buying       1728 non-null   object
 2   maintenance  1728 non-null   object
 3   doors        1728 non-null   object
 4   passengers   1728 non-null   object
 5   boot         1728 non-null   object
 6   safety       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB
None
First five records:
  labels buying maintenance doors passengers   boot safety
0  unacc  vhigh       vhigh     2          2  small    low
1  unacc  vhigh       vhigh     2          2  small    med
2  unacc  vhigh       vhigh     2          2  small   high
3  unacc  vhigh       vhigh     2          2    med    low
4  unacc  vhigh       vhigh     2          2    med    med
Size of X:  (1728, 6)
Size of y:  (1728,)


<IPython.core.display.Javascript object>

In [ ]:
# Insert wrapper here

In [169]:
max_seed_val = 2 ** 32 - 1

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X_cat, y_cat, test_size=0.25, random_state=random.randrange(0, max_seed_val)
)
print("Data has been split.")

Data has been split.


<IPython.core.display.Javascript object>

In [ ]:
# Collectors of values
def collect_current_one_hot_columns(argArray):
    # make list of all values and create steps for them
    return argArray.columns

def collect_current_ordinal_columns(argArray):
    # make list of all values and create steps for them
    return argArray.columns

In [ ]:
# Encode the columns using one hot and ordinal encoders
ohe = OneHotEncoder()
oe = OrdinalEncoder()

# Make column transformer
column_transform = make_column_transformer(
    (ohe, collect_current_one_hot_columns(X)),
    (oe, collect_current_ordinal_columns(X))
)

# Apply transformation
column_transform.fit_transform(X_train, X_test)

In [83]:
# Create a Bayes Classifier
gnb = MultinomialNB()

# Train the model using the training sets
gnb.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = gnb.predict(X_test)

ValueError: could not convert string to float: 'med'

<IPython.core.display.Javascript object>

In [46]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred) * 100, "%")

Accuracy: 97.77777777777777 %


<IPython.core.display.Javascript object>

In [ ]:
"""
useful links:

https://dev.to/codinghappinessweb/analysing-dataset-using-naive-bayes-classifier-3d7o
https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c
https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79
https://towardsdatascience.com/naive-bayes-classifier-how-to-successfully-use-it-in-python-ecf76a995069
https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
"""